In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.join(Path().absolute(), os.pardir, os.pardir)))

In [5]:
from glob import glob
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from tabulate import tabulate
from frequency_response import FrequencyResponse
from constants import ROOT_DIR, HARMAN_ONEAR_PREFERENCE_FREQUENCIES, HARMAN_INEAR_PREFENCE_FREQUENCIES

In [99]:
harman_oe = FrequencyResponse.read_from_csv(os.path.join(ROOT_DIR, 'compensation', 'harman_over-ear_2018.csv'))
errs = []
names = []
i = 0
for fp in glob(os.path.join(ROOT_DIR, 'measurements', 'oratory1990', 'data', 'onear', '*', '*.csv')):
    fr = FrequencyResponse.read_from_csv(fp)
    names.append(fr.name)
    fr.compensate(harman_oe, bass_boost_gain=0.0)
    score, std, slope = fr.harman_onear_preference_score()
    fr.interpolate(HARMAN_ONEAR_PREFERENCE_FREQUENCIES)
    errs.append(np.concatenate([[std, slope, score], fr.raw[::-1]]))
    if i > 9:
        break
    i += 1
    # TODO: ignore samples
errs = np.vstack(errs)
pd.DataFrame(errs.transpose(), columns=names).to_csv('onear_errs.csv', header=True, index=False)

In [98]:
harman_ie = FrequencyResponse.read_from_csv(os.path.join(ROOT_DIR, 'compensation', 'harman_in-ear_2019v2.csv'))
harman_ie.center(1000)
errs = []
names = []
i = 0
for fp in glob(os.path.join(ROOT_DIR, 'measurements', 'oratory1990', 'data', 'inear', '*', '*.csv')):
    fr = FrequencyResponse.read_from_csv(fp)
    names.append(fr.name)
    fr.compensate(harman_ie, bass_boost_gain=0.0)
    score, std, slope, mean = fr.harman_inear_preference_score()
    fr.interpolate(HARMAN_INEAR_PREFENCE_FREQUENCIES)
    errs.append(np.concatenate([[std, slope, mean, score], fr.raw[::-1]]))
    if i > 9:
        break
    i += 1
errs = np.vstack(errs)
pd.DataFrame(errs.transpose(), columns=names).to_csv('inear_errs.csv', header=True, index=False)

In [17]:
np.log2(2)

1.0